# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
# sales = turicreate.SFrame('home_data.sframe/')
types = {
    "bathrooms": float,
    "waterfront": int,
    "sqft_above": int,
    "sqft_living15": float,
    "grade": int,
    "yr_renovated": int,
    "price": float,
    "bedrooms": float,
    "zipcode": str,
    "long": float,
    "sqft_lot15": float,
    "sqft_living": float,
    "floors": str,
    "condition": int,
    "lat": float,
    "date": str,
    "sqft_basement": int,
    "yr_built": int,
    "id": str,
    "sqft_lot": int,
    "view": int,
}
purchases = pd.read_csv("Data/kc_house_data.csv", dtype=types)

In [4]:
purchases.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [5]:
purchases.shape

(21613, 21)

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [ ]:
train_data,test_data = sales.random_split(.8,seed=0)

In [6]:
train = pd.read_csv("Data/kc_house_train_data.csv", parse_dates=['date'], dtype=types)
test = pd.read_csv("Data/kc_house_test_data.csv", parse_dates=['date'], dtype=types)

len(train), len(test)

(17384, 4229)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [18]:
from sklearn.linear_model import LinearRegression

feature_cols = ['sqft_living', 'bedrooms', 'bathrooms']
X = train[feature_cols]
y = train.price

linreg = LinearRegression()

linreg.fit(X, y)

LinearRegression()

In [19]:
intercept_and_paramters = [linreg.intercept_] + list(linreg.coef_)
index = ['Intercept', 'sqft_living', 'bedrooms', 'bathrooms']
pd.DataFrame(intercept_and_paramters, index=index)

,0
Intercept,87912.865815
sqft_living,315.406691
bedrooms,-65081.887116
bathrooms,6942.165986


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [21]:
linreg.predict(train[feature_cols])[0] # should be around 271789

271789.26537996973

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [24]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_pred = model.predict(data)

    # Then compute the residuals/errors
    RSS = outcome - y_pred

    # Then square and add them up
    RSS = np.sum(RSS ** 2)

    return RSS

Test your function by computing the RSS on TEST data for the example model:

In [25]:
get_residual_sum_of_squares(linreg, test[feature_cols], test.price)

273761940583133.75

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

Next create the following 4 new features as column in both TEST and TRAIN data:
* `bedrooms_squared` = `bedrooms` \* `bedrooms`
* `bed_bath_rooms` = `bedrooms`\*`bathrooms`
* `log_sqft_living` = log(`sqft_living`)
* `lat_plus_long` = `lat` + `long` 
As an example here's the first one:

In [30]:
import math

train['bedrooms_squared'] = train.bedrooms.apply(lambda x: x**2)
test['bedrooms_squared'] = test.bedrooms.apply(lambda x: x**2)

train['bed_bath_rooms'] = train.bedrooms * train.bathrooms
test['bed_bath_rooms'] = test.bedrooms * test.bathrooms

train['log_sqft_living'] = train.sqft_living.apply(lambda x: math.log(x))
test['log_sqft_living'] = test.sqft_living.apply(lambda x: math.log(x))

train['lat_plus_long'] = train.lat + train.long
test['lat_plus_long'] = test.lat + test.long

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [42]:
new_feature_cols = [
    "bedrooms_squared",
    "bed_bath_rooms",
    "log_sqft_living",
    "lat_plus_long",
]

test[new_feature_cols].mean()

bedrooms_squared    12.446678
bed_bath_rooms       7.503902
log_sqft_living      7.550275
lat_plus_long      -74.653334
dtype: float64

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [44]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [47]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_1.fit(train[model_1_features], train.price)


model_2 = LinearRegression()
model_2.fit(train[model_2_features], train.price)


model_3 = LinearRegression()
model_3.fit(train[model_3_features], train.price)

LinearRegression()

In [48]:
# Examine/extract each model's coefficients:
pd.DataFrame(model_1.coef_, index=model_1_features)

,0
sqft_living,312.258646
bedrooms,-59586.533154
bathrooms,15706.742083
lat,658619.263931
long,-309374.351268


In [51]:
# Examine/extract each model's coefficients:
pd.DataFrame(model_2.coef_, index=model_2_features)

,0
sqft_living,306.610053
bedrooms,-113446.368070
bathrooms,-71461.308293
lat,654844.629503
long,-294298.969138
bed_bath_rooms,25579.652001


In [50]:
# Examine/extract each model's coefficients:
pd.DataFrame(model_3.coef_, index=model_3_features)

,0
sqft_living,529.422820
bedrooms,34514.229578
bathrooms,67060.781319
lat,534085.610867
long,-406750.710861
bed_bath_rooms,-8570.504395
bedrooms_squared,-6788.586670
log_sqft_living,-561831.484076
lat_plus_long,127334.900006


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

* Answer: Positive

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

* Answer: Negative

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [63]:
rss1 = get_residual_sum_of_squares(model_1, train[model_1_features], train.price)

rss2 = get_residual_sum_of_squares(model_2, train[model_2_features], train.price)

rss3 = get_residual_sum_of_squares(model_3, train[model_3_features], train.price)

pd.DataFrame(
    [rss1, rss2, rss3], index=["Model 1", "Model 2", "Model 3"], columns=["RSS"]
).sort_values("RSS", ascending=True)

,RSS
Model 3,9.034365e+14
Model 2,9.584196e+14
Model 1,9.678800e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

* Answer: Model 3

Combination of all of those feature over-fitted the model 3. So we should expect this model not to generalize well on the test data set.

Now compute the RSS on on TEST data for each of the three models.

In [64]:
rss1 = get_residual_sum_of_squares(model_1, test[model_1_features], test.price)

rss2 = get_residual_sum_of_squares(model_2, test[model_2_features], test.price)

rss3 = get_residual_sum_of_squares(model_3, test[model_3_features], test.price)

pd.DataFrame(
    [rss1, rss2, rss3], index=["Model 1", "Model 2", "Model 3"], columns=["RSS"]
).sort_values("RSS", ascending=True)

,RSS
Model 2,2.233775e+14
Model 1,2.255005e+14
Model 3,2.592363e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

* Answer: Model 2

Model 2 is not as simple as model 1 and at the same time it overcome the over-fit problem with model 3. It's not bad as model 3 and it's better than model 1.

So combining and multiplying multiple feature might be useful but we should we should be cautious while using this method.